<a href="https://colab.research.google.com/github/kimdonggyu2008/audio_synthesis/blob/main/sound_generate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import pickle

import numpy as np
import soundfile as sf

from soundgenerator import SoundGenerator
from sound_autoencoder import VAE
from sound_train import SPECTROGRAMS_PATH

HOP_LENGTH=256
SAVE_DIR_ORIGINAL=""
SAVE_DIR_GENERATED=""
SAVE_DIR_GENERATED=""
MIN_MAX_VALUES_PATH="/content/drive/MyDrive/오디오_합성/"


ModuleNotFoundError: No module named 'soundgenerator'

In [ ]:
def load_fsdd(spectrograms_path):
  x_train=[]
  file_paths=[]
  for root,_,file_names in os.walk(spectrograms_path):#스펙트로그램 가져옴
    for file_name in file_names:
      file_path=os.path.join(root,file_name)#각 파일에 대해서 읽어옴
      spectrograms=np.load(file_path)
      x_train.append(spectrogram)
      file_paths.append(file_path)
  x_train=np.array(x_train) #훈련셋으로 스펙트로그램 입력
  x_train=x_train[...,np.newaxis] #입력된 3차원 스펙트로그램에 1차원 추가
  return x_train,file_paths

def select_spectrograms(spectrograms, #
                        file_paths,
                        min_max_values,
                        num_spectrograms=2):
  sampled_indexes=np.random.choice(range(len(spectrograms)),num_spectrograms)
  sampled_spectrograms=spectrograms[sampled_indexes] #스펙트로그램을 샘플 갯수만큼 자름
  file_paths=[file_paths[index] for index in sampled_indexes]
  sampled_min_max_values=[min_max_values[file_path] for file_path in file_paths]#해당 샘플의 최소 최대 확인
  print(file_paths)
  print(sampled_min_max_values)
  return sampled_spectrogrmas, sampled_min_max_values

def save_signals(signals,save_dir,sample_rate=22050):
  for i,signal in enumerate(signals):
    save_path=os.path.join(save_dir,str(i)+".wav")#웨이브 파일로 저장
    sf.write(save_path,signal,sample_rate)

if __name__=="__main__":
  vae=VAE.load("model")#학습 모델 가져옴
  sound_generator=SoundGenerator(vae,HOP_LENGTH)#생성기 모델 및 학습 거리 지정

  with open(MIN_MAX_VALUES_PATH,"rb") as f:#최소, 최대 데이터 읽어옴
    min_max_values=pickle.load(f)#파일 읽어옴

  specs,file_paths=load_fsdd(SPECTROGRAMS_PATH)#스펙트로그램 파일 읽어옴

  sampled_specs,sampled_min_max_values=select_spectrograms(specs,
                                                           file_paths,
                                                           min_max_values,
                                                           5)

  signals,_=sound_generator.generate(sampled_specs, #샘플 조각, 최소 최댓값을 이용해서 스펙트로그램 생성
                                     sampled_min_max_values)

  original_signals=sound_generator.convert_spectrograms_to_audio(#생성된 스펙트로그램을 오디오로 변경
      sampled_specs,sampled_min_max_values)

  save_signals(signals,SAVE_DIR_GENERATED) #생성된 데이터 저장
  save_signals(original_signals,SAVE_DIR_ORIGINAL) #원 데이터 저장